In [23]:
import random
import numpy
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
random.seed(64)

In [24]:
IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

In [28]:
items = {}
# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-uple.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

In [29]:
creator.create('Fitness',base.Fitness,weights=(-1.0,1.0))
creator.create('Individual',set,fitness=creator.Fitness)
toolbox = base.Toolbox()
# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

C:\Users\AW639XJ\.conda\envs\ey-hindalco-env\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\AW639XJ\.conda\envs\ey-hindalco-env\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [30]:
def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

In [31]:
def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

In [32]:
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

In [33]:
toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [34]:
def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2
    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)
    
    return pop, stats, hof

In [35]:
if __name__ == "__main__":
    main()

gen	nevals	avg                        	std                      	min                        	max                        
0  	50    	[ 28.48       241.55944152]	[ 6.78893217 58.23449577]	[ 10.         109.69179614]	[ 41.         377.87971041]
1  	87    	[ 15.08       170.65437179]	[ 16.32279388 163.94781653]	[0. 0.]                    	[ 48.         443.60115511]
2  	91    	[ 8.06      99.1848964]    	[ 14.24838236 161.59868545]	[0. 0.]                    	[ 42.         443.60115511]
3  	90    	[ 4.18       66.30838345]  	[  9.86243378 132.13037076]	[0. 0.]                    	[ 42.         443.60115511]
4  	92    	[ 3.84       62.63986028]  	[  8.83031143 128.01301331]	[0. 0.]                    	[ 42.         443.60115511]
5  	87    	[ 5.24       89.81112688]  	[  9.72534832 153.55612269]	[0. 0.]                    	[ 33.         459.73815613]
6  	82    	[  7.04       126.80913652]	[ 10.4076126  167.56009711]	[0. 0.]                    	[ 33.         459.73815613]
7  	90    	[  7.32  